### Reading File 

In [1]:
import pandas as pd

In [2]:
dataFrame = pd.read_csv('HIST_PAINEL_COVIDBR_04mai2021.csv', sep=';')

### Noisy Data

<span style="color:#3385ff"> **Removing noisy regions and set Index**</span>

In [3]:
dataFrame = dataFrame[dataFrame['regiao'] != 'Brasil']

In [4]:
dataFrame.set_index('regiao', inplace=True)

<span style='color:#3385ff'> **Removing noisy municipalities** </span>

In [5]:
dataFrame['municipiosTemp'] = dataFrame['municipio'].notna()

In [6]:
# Saving data to compare the regions later.
municipalitiesNan = dataFrame[dataFrame['municipiosTemp'] == False]

In [7]:
dataFrame = dataFrame[dataFrame['municipiosTemp']]

In [8]:
del dataFrame['municipiosTemp']

 <span style='color:#3385ff'> **Removing noisy New recovered and New in follow-ups** </span>

In [9]:
dataFrame['RecuperadosnovosTemp'] = dataFrame['Recuperadosnovos'].notna()

In [10]:
dataFrame[dataFrame['RecuperadosnovosTemp']].shape[0]

0

In [11]:
dataFrame['emAcompanhamentoNovosTemp'] = dataFrame['emAcompanhamentoNovos'].notna()

In [12]:
dataFrame[dataFrame['emAcompanhamentoNovosTemp']].shape[0]

0

In [13]:
del dataFrame['RecuperadosnovosTemp']
del dataFrame['emAcompanhamentoNovosTemp']

del dataFrame['Recuperadosnovos']
del dataFrame['emAcompanhamentoNovos']

## Data Analyzing

In [14]:
dataFrame

,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,interior/metropolitana
regiao,,,,,,,,,,,,,,
Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,ZONA DA MATA,2020-03-27,13,22945.0,0,0,0,0,0.0
Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,ZONA DA MATA,2020-03-28,13,22945.0,0,0,0,0,0.0
Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,ZONA DA MATA,2020-03-29,14,22945.0,0,0,0,0,0.0
Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,ZONA DA MATA,2020-03-30,14,22945.0,0,0,0,0,0.0
Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,ZONA DA MATA,2020-03-31,14,22945.0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Centro-Oeste,DF,Brasília,53,530010.0,53001.0,DISTRITO FEDERAL,2021-04-30,17,3015268.0,378783,1127,7798,37,1.0
Centro-Oeste,DF,Brasília,53,530010.0,53001.0,DISTRITO FEDERAL,2021-05-01,17,3015268.0,379752,969,7826,28,1.0
Centro-Oeste,DF,Brasília,53,530010.0,53001.0,DISTRITO FEDERAL,2021-05-02,18,3015268.0,380639,887,7855,29,1.0


In [15]:
temp = dataFrame[dataFrame['data'] == '2021-05-01']
temp.reset_index()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,interior/metropolitana
0,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,ZONA DA MATA,2021-05-01,17,22945.0,3429,13,55,1,0.0
1,Norte,RO,Ariquemes,11,110002.0,11001.0,VALE DO JAMARI,2021-05-01,17,107863.0,18387,97,396,3,0.0
2,Norte,RO,Cabixi,11,110003.0,11006.0,CONE SUL,2021-05-01,17,5312.0,636,0,14,0,0.0
3,Norte,RO,Cacoal,11,110004.0,11002.0,CAFE,2021-05-01,17,85359.0,11275,-27,205,0,0.0
4,Norte,RO,Cerejeiras,11,110005.0,11006.0,CONE SUL,2021-05-01,17,16323.0,2069,7,49,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,Centro-Oeste,GO,Vianópolis,52,522200.0,52002.0,CENTRO SUL,2021-05-01,17,13863.0,606,0,16,0,0.0
5566,Centro-Oeste,GO,Vicentinópolis,52,522205.0,52002.0,CENTRO SUL,2021-05-01,17,8743.0,1056,6,18,1,0.0
5567,Centro-Oeste,GO,Vila Boa,52,522220.0,52003.0,ENTORNO NORTE,2021-05-01,17,6171.0,124,1,4,0,0.0
5568,Centro-Oeste,GO,Vila Propício,52,522230.0,52018.0,SAO PATRICIO II,2021-05-01,17,5821.0,313,0,3,0,0.0


In [16]:
temp = temp.groupby('estado')

In [18]:
temp.describe()

coduf                                          codmun                 \
        count  mean  std   min   25%   50%   75%   max  count           mean   
estado                                                                         
AC       22.0  12.0  0.0  12.0  12.0  12.0  12.0  12.0   22.0  120034.636364   
AL      102.0  27.0  0.0  27.0  27.0  27.0  27.0  27.0  102.0  270477.901961   
AM       62.0  13.0  0.0  13.0  13.0  13.0  13.0  13.0   62.0  130216.419355   
AP       16.0  16.0  0.0  16.0  16.0  16.0  16.0  16.0   16.0  160036.500000   
BA      417.0  29.0  0.0  29.0  29.0  29.0  29.0  29.0  417.0  291714.592326   
CE      184.0  23.0  0.0  23.0  23.0  23.0  23.0  23.0  184.0  230703.440217   
DF        1.0  53.0  NaN  53.0  53.0  53.0  53.0  53.0    1.0  530010.000000   
ES       78.0  32.0  0.0  32.0  32.0  32.0  32.0  32.0   78.0  320290.205128   
GO      246.0  52.0  0.0  52.0  52.0  52.0  52.0  52.0  246.0  521133.186992   
MA      217.0  21.0  0.0  21.0  21.0  21.0  21.0  21.0  217.0  210646.640553   
MG      853.0  31.0  0.0  31.0  31.0  31.0  31.0  31.0  853.0  313697.720985   
MS       79.0  50.0  0.0  50.0  50.0  50.0  50.0  50.0   79.0  500449.189873   
MT      141.0  51.0  0.0  51.0  51.0  51.0  51.0  51.0  141.0  510524.085106   
PA      144.0  15.0  0.0  15.0  15.0  15.0  15.0  15.0  144.0  150446.479167   
PB      223.0  25.0  0.0  25.0  25.0  25.0  25.0  25.0  223.0  250869.973094   
PE      185.0  26.0  0.0  26.0  26.0  26.0  26.0  26.0  185.0  260823.702703   
PI      224.0  22.0  0.0  22.0  22.0  22.0  22.0  22.0  224.0  220568.919643   
PR      399.0  41.0  0.0  41.0  41.0  41.0  41.0  41.0  399.0  411443.197995   
RJ       92.0  33.0  0.0  33.0  33.0  33.0  33.0  33.0   92.0  330312.380435   
RN      167.0  24.0  0.0  24.0  24.0  24.0  24.0  24.0  167.0  240764.191617   
RO       52.0  11.0  0.0  11.0  11.0  11.0  11.0  11.0   52.0  110067.711538   
RR       15.0  14.0  0.0  14.0  14.0  14.0  14.0  14.0   15.0  140030.800000   
RS      497.0  43.0  0.0  43.0  43.0  43.0  43.0  43.0  497.0  431187.289738   
SC      295.0  42.0  0.0  42.0  42.0  42.0  42.0  42.0  295.0  420980.908475   
SE       75.0  28.0  0.0  28.0  28.0  28.0  28.0  28.0   75.0  280392.293333   
SP      645.0  35.0  0.0  35.0  35.0  35.0  35.0  35.0  645.0  352869.334884   
TO      139.0  17.0  0.0  17.0  17.0  17.0  17.0  17.0  139.0  171154.446043   

        ... obitosNovos        interior/metropolitana                      \
        ...         75%    max                  count      mean       std   
estado  ...                                                                 
AC      ...         0.0    3.0                   22.0  0.045455  0.213201   
AL      ...         0.0    7.0                  102.0  0.117647  0.323781   
AM      ...         0.0    2.0                   62.0  0.209677  0.410402   
AP      ...         0.0    6.0                   16.0  0.187500  0.403113   
BA      ...         0.0   22.0                  417.0  0.031175  0.173999   
CE      ...         1.0  126.0                  184.0  0.103261  0.305130   
DF      ...        28.0   28.0                    1.0  1.000000       NaN   
ES      ...         0.0    5.0                   78.0  0.089744  0.287664   
GO      ...         0.0   16.0                  246.0  0.085366  0.279995   
MA      ...         0.0    7.0                  217.0  0.059908  0.237865   
MG      ...         0.0   45.0                  853.0  0.039859  0.195743   
MS      ...         0.0   21.0                   79.0  0.012658  0.112509   
MT      ...         0.0    7.0                  141.0  0.042553  0.202567   
PA      ...         0.0   27.0                  144.0  0.048611  0.215804   
PB      ...         0.0   11.0                  223.0  0.053812  0.226153   
PE      ...         0.0   21.0                  185.0  0.081081  0.273700   
PI      ...         0.0   11.0                  224.0  0.058036  0.234335   
PR      ...         0.0   48.0                  399.0  0.072682  0.

In [19]:
temp['casosAcumulado'].sum()

estado
AC      78016
AL     174476
AM     371306
AP     105971
BA     895046
CE     668796
DF     379752
ES     434513
GO     552461
MA     267548
MG    1362336
MS     249033
MT     361209
PA     472986
PB     293717
PE     408763
PI     240868
PR     946173
RJ     745456
RN     224245
RO     213117
RR      94043
RS     979807
SC     890577
SE     203036
SP    2917411
TO     160449
Name: casosAcumulado, dtype: int64